In [1]:
# !pip install scikit-image

In [1]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import namedtuple

2023-08-31 11:52:46.009425: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 11:52:46.033209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 11:52:46.396598: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### MY_Generator

In [12]:
# class MY_Generator(tf.keras.utils.Sequence):

#     def __init__(self, image_filenames, labels, batch_size):
#         self.image_filenames, self.labels = image_filenames, labels
#         self.batch_size = batch_size

#     def __len__(self):
#         return np.ceil(len(self.image_filenames) / float(self.batch_size))

#     def __getitem__(self, idx):
#         batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

#         return np.array([
#             resize(imread(file_name), (200, 200))
#                for file_name in batch_x]), np.array(batch_y)

In [ ]:
# my_training_batch_generator = My_Generator(training_filenames, GT_training, batch_size)

### Explore label ids

In [4]:
raw_data = pd.read_csv("data_from_json.csv", sep='\t', encoding='utf-8', index_col=0)

In [5]:
raw_data.head()

,label,polygon,city,pic_id,set_type,target,label_id
0,sky,"[[126, 9], [235, 182], [797, 315], [1045, 339]...",lindau,000057_000019,val,sky,5
1,road,"[[37, 561], [362, 514], [811, 398], [955, 361]...",lindau,000057_000019,val,flat,1
2,building,"[[1140, 324], [1139, 334], [1138, 337], [1123,...",lindau,000057_000019,val,construction,2
3,vegetation,"[[1129, 376], [1127, 371], [1127, 369], [1125,...",lindau,000057_000019,val,nature,4
4,building,"[[1073, 337], [1072, 294], [1055, 291], [1049,...",lindau,000057_000019,val,construction,2


### Third

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
import tqdm
import cv2
from tensorflow.python.keras.callbacks import Callback

In [7]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

In [8]:
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}

In [9]:
image_dir = 'P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train/aachen'
mask_dir = 'P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen'
image_list = os.listdir(image_dir)
mask_list = os.listdir(mask_dir)
mask_list = [x for x in mask_list if x.endswith("labelIds.png")]
image_list.sort()
mask_list.sort()
print(f'. . . . .Number of images: {len(image_list)}\n. . . . .Number of masks: {len(mask_list)}')

# sanity check
# for i in range(len(image_list)):
#     assert image_list[i][16:] == mask_list[i][24:]


batch_size = 16
samples = 50000
steps = samples//batch_size
img_height, img_width = 256, 256
classes = 8
filters_n = 64

. . . . .Number of images: 174
. . . . .Number of masks: 174


In [10]:
class seg_gen(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.randint(0, 50000, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        for i in idx:
            _image = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}', target_size=(img_height, img_width)))/255.   
            img = image.img_to_array(image.load_img(f'{mask_dir}/{mask_list[i]}', grayscale=True, target_size=(img_height, img_width)))
            labels = np.unique(img)
            if len(labels) < 3:
                idx = np.random.randint(0, 50000, batch_size-drawn)
                continue
            img = np.squeeze(img)
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height*img_width, 8))
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1
        return np.array(batch_x), np.array(batch_y)

In [11]:
class visualize(Callback):
    def on_epoch_end(self, epoch, logs):
        print(f'\nGenerating output at epoch : {epoch}')
        i = 567
        img = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}'))/255.    
        dims = img.shape
        x = cv2.resize(img, (256, 256))
        x = np.float32(x)/255.
        z = unet.predict(np.expand_dims(x, axis=0))
        z = np.squeeze(z)
        z = z.reshape(256, 256, 8)
        z = cv2.resize(z, (dims[1], dims[0]))
        y = np.argmax(z, axis=2)
        
        construction = np.zeros_like(y)
        human = np.zeros_like(y)
        vehicle = np.zeros_like(y)
        construction[y==2] = 255.
        human[y==6] = 255.
        vehicle[y==7] = 255.
        
        result = img.copy()
        alpha = 0.4
        img[:,:,1] = construction
        img[:,:,2] = vehicle 
        img[:,:,0] = human

        cv2.addWeighted(img, alpha, result, 1-alpha, 0, result)
        cv2.imwrite(f'outputs/{epoch}.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        print('Wrote file to disk')

In [12]:
# train_gen = seg_gen(image_list, mask_list, batch_size)

In [19]:
# train_gen

In [26]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [28]:
def DilatedNet(img_height, img_width, nclasses, use_ctx_module=False, bn=False):
    print('. . . . .Building DilatedNet. . . . .')
    def bilinear_upsample(image_tensor):
        # upsampled = tf.image.resize_bilinear(image_tensor, size=(img_height, img_width))
        upsampled = tf.compat.v1.image.resize_bilinear(image_tensor, size=(img_height, img_width))
        return upsampled
    
    def conv_block(conv_layers, tensor, nfilters, size=3, name='', padding='same', dilation_rate=1,pool=False):
        if dilation_rate == 1:
            conv_type = 'conv'
        else:
            conv_type = 'dilated_conv'
        for i in range(conv_layers):
            tensor = Conv2D(nfilters, size, padding=padding, use_bias=False, dilation_rate=dilation_rate, name=f'block{name}_{conv_type}{i+1}')(tensor)
            if bn:
                tensor = BatchNormalization(name=f'block{name}_bn{i+1}')(tensor)
            tensor = Activation('relu', name=f'block{name}_relu{i+1}')(tensor)
        if pool:
            tensor = MaxPooling2D(2, name=f'block{name}_pool')(tensor)
        return tensor
       
    nfilters = 64
    img_input = Input(shape=(img_height, img_width, 3))
    x = conv_block(conv_layers=2,tensor=img_input, nfilters=nfilters*1, size=3, pool=True, name=1)
    x = conv_block(conv_layers=2,tensor=x, nfilters=nfilters*2, size=3, pool=True, name=2)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*4, size=3, pool=True, name=3)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*8, size=3, name=4)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*8, size=3,dilation_rate=2, name=5)
    x = conv_block(conv_layers=1,tensor=x, nfilters=nfilters*64, size=7,dilation_rate=4, name='_FCN1')
    x = Dropout(0.5)(x)
    x = conv_block(conv_layers=1,tensor=x, nfilters=nfilters*64, size=1, name='_FCN2')
    x = Dropout(0.5)(x)  
    x = Conv2D(filters=nclasses, kernel_size=1, padding='same', name=f'frontend_output')(x)
    if use_ctx_module:
        x = conv_block(conv_layers=2, tensor=x, nfilters=nclasses*2, size=3, name='_ctx1')
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*4, size=3, name='_ctx2', dilation_rate=2)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*8, size=3, name='_ctx3', dilation_rate=4)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*16, size=3, name='_ctx4', dilation_rate=8)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*32, size=3, name='_ctx5', dilation_rate=16)        
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*32, size=3, name='_ctx7')
        x = Conv2D(filters=nclasses, kernel_size=1, padding='same', name=f'ctx_output')(x)
    x = Lambda(bilinear_upsample, name='bilinear_upsample')(x)
    x = Reshape((img_height*img_width, nclasses))(x)
    x = Activation('softmax', name='final_softmax')(x)
  
    model = Model(inputs=img_input, outputs=x, name='DilatedNet')
    print('. . . . .Building network successful. . . . .')
    return model

In [29]:
unet = DilatedNet(256, 256, 8,use_ctx_module=True, bn=True)
# p_unet = multi_gpu_model(unet, 4)
# p_unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
tb = TensorBoard(log_dir='logs', write_graph=True)
mc = ModelCheckpoint(mode='max', filepath='models-dr/pdilated.h5', monitor='acc', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='acc', patience=6, verbose=1)
vis = visualize()
callbacks = [tb, mc, es]
train_gen = seg_gen(image_list, mask_list, batch_size)

. . . . .Building DilatedNet. . . . .
. . . . .Building network successful. . . . .


2023-08-31 12:35:18.121067: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-08-31 12:35:18.121085: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2023-08-31 12:35:18.122607: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1671] Profiler found 1 GPUs
2023-08-31 12:35:18.248332: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2023-08-31 12:35:18.248544: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1805] CUPTI activity buffer flushed


In [30]:
# # p_unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
print('Saving final weights')
unet.save_weights('dilated.h5')

/tmp/ipykernel_42823/1447523711.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)


IndexError: list index out of range

### Second

In [14]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            # X[i,] = np.load('data/' + ID + '.npy')
            img_path = os.path.join('P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/' + ID + '.png')
            X[i,] = np.array(Image.open(img_path))
            
            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [20]:
import numpy as np

from keras.models import Sequential

# Parameters
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 8,
          'n_channels': 1,
          'shuffle': True}

# Datasets
# partition = # IDs
# labels = # Labels

# Generators

training_generator = DataGenerator(image_list, mask_list, **params)
# validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
# model = Sequential()
# [...] # Architecture
# model.compile()

# # Train model on dataset
# model.fit_generator(generator=training_generator,
#                     validation_data=validation_generator,
#                     use_multiprocessing=True,
#                     workers=6)

In [21]:
training_generator